# Latin Difficulty Clustering
**(CL/AI Task III)**


In [3]:
from cltk import NLP
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


# Tag Sentences

In [ ]:
tagged_sentences = []

with open('corpus.txt', 'r') as f:
	corpus = f.read()

# corpus = "Quo usque tandem abutere, Catilina, patientia nostra? quam diu etiam furor iste tuus nos eludet?"

cltk_nlp = NLP(language="lat")
cltk_doc = cltk_nlp.analyze(text=corpus)

for sentence in cltk_doc.sentences:
    sent_pos = ""
    sent_clean = ""
    for word in sentence.words:
        if(sent_pos == ""):
            sent_pos += word.pos.name
            sent_clean += word.string
        else:
            sent_pos += " " + word.pos.name
            sent_clean += " " + word.string
    sent_obj = {
        "sentence": sent_clean, 
        "pos":sent_pos
    }
    tagged_sentences.append(sent_obj)
    print(sent_clean)

df = pd.DataFrame(tagged_sentences)


# Clustering

In [ ]:
### Get all the features columns except the class
features = list(_data.columns)[:-2]

### Get the features data
data = _data[features]

clustering_kmeans = KMeans(n_clusters=2, precompute_distances="auto", n_jobs=-1)
data['clusters'] = clustering_kmeans.fit_predict(data)

### Run PCA on the data and reduce the dimensions in pca_num_components dimensions

pca_num_components = 2

reduced_data = PCA(n_components=pca_num_components).fit_transform(data)
results = pd.DataFrame(reduced_data,columns=['pca1','pca2'])

sns.scatterplot(x="pca1", y="pca2", hue=data['clusters'], data=results)
plt.title('K-means Clustering with 2 dimensions')
plt.show()